Generate a series of short-term signals from the book and trade data of a fixed 10-minute window to predict the realized volatility of the next 10-minute window

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob

In [3]:
# Data directory
data_dir = '../input/optiver-realized-volatility-prediction/'

# Functions for preprocessing

In [4]:
# To calculate first WAP
def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap

# To calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap

# To calculate log return
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

# To calculate realized volatility 
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# To count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# Book data

In [5]:
# Load parquet
book_train = pd.read_parquet(data_dir + "book_train.parquet/stock_id=15")

# Book data snapshot
book_train.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,0.999519,0.999839,0.999454,0.999904,2,166,2,12
1,5,1,0.999711,1.000225,0.999647,1.000289,100,20,100,20
2,5,2,0.999775,1.000225,0.999711,1.000289,1,20,400,20
3,5,3,0.999839,1.000225,0.999775,1.000289,100,20,1,20
4,5,4,0.999839,1.000225,0.999711,1.000289,1,20,400,20


# Functions to preprocess book data

In [6]:
# To preprocess book data (for each stock_id)
def preprocessor_book(file_path):
    df = pd.read_parquet(file_path)
    
    # Calculate WAP
    df['wap'] = calc_wap(df)
    df['wap2'] = calc_wap2(df)
    
    # Calculate log return
    df['log_return'] = df.groupby('time_id')['wap'].apply(log_return)
    df['log_return2'] = df.groupby('time_id')['wap2'].apply(log_return)
    
    # Calculate WAP balance
    df['wap_balance'] = abs(df['wap'] - df['wap2'])
    
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1'])/2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))

    # Dict for aggregations
    create_feature_dict = {
        'wap': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread': [np.sum, np.mean, np.std],
        'bid_spread': [np.sum, np.mean, np.std],
        'ask_spread': [np.sum, np.mean, np.std],
        'total_volume': [np.sum, np.mean, np.std],
        'volume_imbalance': [np.sum, np.mean, np.std]
    }
    
    # Functions to get group stats for different time windows (0-199, 200-399, 400-599)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by time window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns] #time_id is changed to time_id_
        # Add a suffix to differentiate time windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get stats for 3 time windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    
    # Merge them all
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    
    # Drop unecessary time_id__200 and time_id__400
    df_feature.drop(['time_id__200', 'time_id__400'], axis = 1, inplace = True)
    
    # Create row_id to merge book and trade data
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    
    return df_feature

In [7]:
%%time
file_path = data_dir + "book_train.parquet/stock_id=0"
preprocessor_book(file_path)

CPU times: user 14 s, sys: 462 ms, total: 14.4 s
Wall time: 14.5 s


,wap_sum,wap_mean,wap_std,wap2_sum,wap2_mean,wap2_std,log_return_sum,log_return_realized_volatility,log_return_mean,log_return_std,...,ask_spread_sum_400,ask_spread_mean_400,ask_spread_std_400,total_volume_sum_400,total_volume_mean_400,total_volume_std_400,volume_imbalance_sum_400,volume_imbalance_mean_400,volume_imbalance_std_400,row_id
0,303.125061,1.003725,0.000693,303.105530,1.003661,0.000781,0.002292,0.004499,7.613697e-06,0.000260,...,-0.018721,-0.000191,0.000140,25724,262.489796,118.188932,12184,124.326531,82.090066,0-5
1,200.047775,1.000239,0.000262,200.041168,1.000206,0.000272,0.000360,0.001204,1.810376e-06,0.000086,...,-0.009736,-0.000133,0.000063,35040,480.000000,167.075582,7018,96.136986,79.708203,0-11
2,187.913849,0.999542,0.000864,187.939819,0.999680,0.000862,-0.002074,0.002369,-1.109168e-05,0.000173,...,-0.010387,-0.000204,0.000164,23154,454.000000,115.120632,7778,152.509804,100.093231,0-16
3,119.859779,0.998832,0.000757,119.835945,0.998633,0.000656,-0.002828,0.002574,-2.376725e-05,0.000236,...,-0.001111,-0.000048,0.000010,11476,498.956522,156.965682,3538,153.826087,95.507569,0-31
4,175.932861,0.999619,0.000258,175.934250,0.999626,0.000317,-0.000002,0.001894,-1.021740e-08,0.000144,...,-0.006946,-0.000126,0.000083,18697,339.945455,129.296590,4709,85.618182,93.493413,0-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,296.387482,0.997938,0.000747,296.365479,0.997864,0.000769,-0.002136,0.002579,-7.215169e-06,0.000150,...,-0.017949,-0.000197,0.000118,28874,317.296703,145.085683,20098,220.857143,151.670298,0-32751
3826,206.063904,1.000310,0.000551,206.100403,1.000487,0.000599,0.000403,0.002206,1.966603e-06,0.000154,...,-0.011113,-0.000192,0.000109,32829,566.017241,215.469255,14539,250.672414,152.685480,0-32753
3827,187.915695,0.999552,0.000743,187.897705,0.999456,0.000736,0.001663,0.002913,8.895233e-06,0.000213,...,-0.006299,-0.000082,0.000050,25816,335.272727,117.648074,8412,109.246753,88.450872,0-32758
3828,307.723694,1.002357,0.000356,307.732635,1.002386,0.000424,0.000520,0.003046,1.698934e-06,0.000174,...,-0.017453,-0.000169,0.000172,44822,435.165049,165.821490,11920,115.728155,89.854042,0-32763


# Trade data

In [8]:
# Load parquet
trade_train = pd.read_parquet(data_dir + "trade_train.parquet/stock_id=0")

# Trade data snapshot
trade_train.head()

,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1


## Functions to preprocess trade data

In [9]:
# To preprocess trade data (for each stock_id)
def preprocessor_trade(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.mean]
    }
    
    # Functions to get group stats for different time windows (0-199, 200-399, 400-599)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by time window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns] #time_id is changed to time_id_
        # Add a suffix to differentiate time windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get stats for 3 time windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_200 = get_stats_window(seconds_in_bucket = 200, add_suffix = True)
    df_feature_400 = get_stats_window(seconds_in_bucket = 400, add_suffix = True)
    
    # Merge them all
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    
    # Drop unecessary time_id__200 and time_id__400
    df_feature.drop(['time_id__200', 'time_id__400'], axis = 1, inplace = True)
    
    # Add prefix trade_
    df_feature = df_feature.add_prefix('trade_')
    
    # Create row_id to merge with book and trade data
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    
    return df_feature

In [10]:
%%time
file_path = data_dir + "trade_train.parquet/stock_id=0"
preprocessor_trade(file_path)

CPU times: user 6.58 s, sys: 183 ms, total: 6.76 s
Wall time: 6.52 s


,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_200,trade_seconds_in_bucket_count_unique_200,trade_size_sum_200,trade_order_count_mean_200,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_count_unique_400,trade_size_sum_400,trade_order_count_mean_400,row_id
0,0.002006,40,3179,2.750000,0.001666,27.0,1901.0,2.555556,0.001121,16.0,1045.0,2.437500,0-5
1,0.000901,30,1289,1.900000,0.000802,22.0,1124.0,2.045455,0.000510,11.0,829.0,2.090909,0-11
2,0.001961,25,2161,2.720000,0.001575,18.0,1691.0,2.833333,0.001048,10.0,1087.0,3.400000,0-16
3,0.001561,15,1962,3.933333,0.001090,10.0,1561.0,4.700000,0.000802,3.0,514.0,3.666667,0-31
4,0.000871,22,1791,4.045455,0.000498,14.0,1458.0,4.428571,0.000395,6.0,162.0,3.666667,0-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0.001519,52,3450,3.057692,0.001257,39.0,2407.0,3.128205,0.000911,28.0,1856.0,3.142857,0-32751
3826,0.001411,28,4547,3.892857,0.001235,18.0,2493.0,3.555556,0.000765,6.0,1401.0,5.166667,0-32753
3827,0.001521,36,4250,3.500000,0.001243,23.0,2295.0,3.608696,0.000875,13.0,1149.0,2.692308,0-32758
3828,0.001794,53,3217,2.150943,0.001622,33.0,2171.0,2.030303,0.001070,16.0,1463.0,2.312500,0-32763


# Functions to get group stats for the stock_id and time_id

In [12]:
def get_time_stock(df):
    # Get realized volatility columns
    vol_cols = ['log_return_realized_volatility', 'log_return2_realized_volatility', 
                'log_return_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'log_return_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'trade_log_return_realized_volatility', 
                'trade_log_return_realized_volatility_200', 
                'trade_log_return_realized_volatility_400'
               ]

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    
    # Drop unecessary columns
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    
    return df

# Combined preprocessor function

In [13]:
# To make preprocessing function in parallel (for each stock_id)
def preprocessor(list_stock_ids, is_train = True):
    from joblib import Parallel, delayed # parallel computing to save time
    df = pd.DataFrame()
    
    # Parallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
        # Preprocess book & trade data and merge them    
        df_tmp = pd.merge(preprocessor_book(file_path_book), preprocessor_trade(file_path_trade), on = 'row_id', how = 'left')
        # Return the merged DataFrame
        return pd.concat([df,df_tmp])
    
    # Use parallel API to call parallel for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatinate all the DataFrames that return from parallel
    df = pd.concat(df, ignore_index = True)
    
    return df

In [14]:
list_stock_ids = [0,1]
preprocessor(list_stock_ids, is_train = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   35.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   35.5s finished


,wap_sum,wap_mean,wap_std,wap2_sum,wap2_mean,wap2_std,log_return_sum,log_return_realized_volatility,log_return_mean,log_return_std,...,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_200,trade_seconds_in_bucket_count_unique_200,trade_size_sum_200,trade_order_count_mean_200,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_count_unique_400,trade_size_sum_400,trade_order_count_mean_400
0,303.125061,1.003725,0.000693,303.105530,1.003661,0.000781,0.002292,0.004499,7.613697e-06,0.000260,...,3179,2.750000,0.001666,27.0,1901.0,2.555556,0.001121,16.0,1045.0,2.437500
1,200.047775,1.000239,0.000262,200.041168,1.000206,0.000272,0.000360,0.001204,1.810376e-06,0.000086,...,1289,1.900000,0.000802,22.0,1124.0,2.045455,0.000510,11.0,829.0,2.090909
2,187.913849,0.999542,0.000864,187.939819,0.999680,0.000862,-0.002074,0.002369,-1.109168e-05,0.000173,...,2161,2.720000,0.001575,18.0,1691.0,2.833333,0.001048,10.0,1087.0,3.400000
3,119.859779,0.998832,0.000757,119.835945,0.998633,0.000656,-0.002828,0.002574,-2.376725e-05,0.000236,...,1962,3.933333,0.001090,10.0,1561.0,4.700000,0.000802,3.0,514.0,3.666667
4,175.932861,0.999619,0.000258,175.934250,0.999626,0.000317,-0.000002,0.001894,-1.021740e-08,0.000144,...,1791,4.045455,0.000498,14.0,1458.0,4.428571,0.000395,6.0,162.0,3.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7655,307.043610,1.000142,0.000396,307.001831,1.000006,0.000372,-0.000920,0.003723,-3.007318e-06,0.000213,...,3249,2.775510,0.001462,31.0,2290.0,3.129032,0.001028,16.0,1532.0,3.937500
7656,497.706604,1.007503,0.006260,497.615204,1.007318,0.006176,0.014279,0.010829,2.896253e-05,0.000487,...,75903,7.874317,0.007641,126.0,49265.0,8.293651,0.003885,51.0,19713.0,6.862745
7657,313.267395,1.000854,0.000564,313.265869,1.000849,0.000578,0.001721,0.003135,5.514881e-06,0.000178,...,2239,2.615385,0.001692,20.0,1734.0,2.700000,0.001355,7.0,654.0,2.714286
7658,435.315918,1.003032,0.001669,435.319824,1.003041,0.001698,0.003910,0.003750,9.030276e-06,0.000180,...,16648,2.935780,0.002478,76.0,10960.0,2.710526,0.001684,37.0,5279.0,2.837838


# **Change data type to reduce memory usage**

In [15]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [16]:
%%time
train = reduce_mem_usage(pd.read_csv(data_dir + 'train.csv'))
#test = reduce_mem_usage(pd.read_csv(data_dir + 'test.csv'))
print("Shape of train set: ",train.shape)
#print("Shape of test set: ",test.shape)

Mem. usage decreased to  2.86 Mb (70.8% reduction)
Shape of train set:  (428932, 3)
CPU times: user 137 ms, sys: 26.2 ms, total: 163 ms
Wall time: 271 ms


# Training set

In [17]:
train_ids = train.stock_id.unique()

In [18]:
%%time
df_train = preprocessor(list_stock_ids = train_ids, is_train = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed: 32.0min finished


CPU times: user 6.76 s, sys: 728 ms, total: 7.49 s
Wall time: 32min 3s


In [19]:
#Combine stock_id and time_id
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]

df_train = train.merge(df_train, on = ['row_id'], how = 'left')

In [20]:
df_train.head()

,row_id,target,wap_sum,wap_mean,wap_std,wap2_sum,wap2_mean,wap2_std,log_return_sum,log_return_realized_volatility,...,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_200,trade_seconds_in_bucket_count_unique_200,trade_size_sum_200,trade_order_count_mean_200,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_count_unique_400,trade_size_sum_400,trade_order_count_mean_400
0,0-5,0.004135,303.125061,1.003725,0.000693,303.105530,1.003661,0.000781,0.002292,0.004499,...,3179.0,2.750000,0.001666,27.0,1901.0,2.555556,0.001121,16.0,1045.0,2.437500
1,0-11,0.001445,200.047775,1.000239,0.000262,200.041168,1.000206,0.000272,0.000360,0.001204,...,1289.0,1.900000,0.000802,22.0,1124.0,2.045455,0.000510,11.0,829.0,2.090909
2,0-16,0.002169,187.913849,0.999542,0.000864,187.939819,0.999680,0.000862,-0.002074,0.002369,...,2161.0,2.720000,0.001575,18.0,1691.0,2.833333,0.001048,10.0,1087.0,3.400000
3,0-31,0.002195,119.859779,0.998832,0.000757,119.835945,0.998633,0.000656,-0.002828,0.002574,...,1962.0,3.933333,0.001090,10.0,1561.0,4.700000,0.000802,3.0,514.0,3.666667
4,0-62,0.001747,175.932861,0.999619,0.000258,175.934250,0.999626,0.000317,-0.000002,0.001894,...,1791.0,4.045455,0.000498,14.0,1458.0,4.428571,0.000395,6.0,162.0,3.666667


# Test set

In [21]:
test = pd.read_csv(data_dir + 'test.csv')

In [22]:
test_ids = test.stock_id.unique()

In [23]:
%%time
df_test = preprocessor(list_stock_ids = test_ids, is_train = False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


CPU times: user 26 ms, sys: 3.66 ms, total: 29.7 ms
Wall time: 274 ms


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.3s finished


In [24]:
df_test = test.merge(df_test, on = ['row_id'], how = 'left')

## Target encoding by stock_id

In [25]:
from sklearn.model_selection import KFold
#stock_id target encoding
df_train['stock_id'] = df_train['row_id'].apply(lambda x:x.split('-')[0])
df_test['stock_id'] = df_test['row_id'].apply(lambda x:x.split('-')[0])

stock_id_target_mean = df_train.groupby('stock_id')['target'].mean() 
df_test['stock_id_target_enc'] = df_test['stock_id'].map(stock_id_target_mean) # test_set

#training
tmp = np.repeat(np.nan, df_train.shape[0])
kf = KFold(n_splits = 20, shuffle = True,random_state = 99)
for idx_1, idx_2 in kf.split(df_train):
    target_mean = df_train.iloc[idx_1].groupby('stock_id')['target'].mean()

    tmp[idx_2] = df_train['stock_id'].iloc[idx_2].map(target_mean)
df_train['stock_id_target_enc'] = tmp

## Model Building

In [26]:
df_train.head()

,row_id,target,wap_sum,wap_mean,wap_std,wap2_sum,wap2_mean,wap2_std,log_return_sum,log_return_realized_volatility,...,trade_log_return_realized_volatility_200,trade_seconds_in_bucket_count_unique_200,trade_size_sum_200,trade_order_count_mean_200,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_count_unique_400,trade_size_sum_400,trade_order_count_mean_400,stock_id,stock_id_target_enc
0,0-5,0.004135,303.125061,1.003725,0.000693,303.105530,1.003661,0.000781,0.002292,0.004499,...,0.001666,27.0,1901.0,2.555556,0.001121,16.0,1045.0,2.437500,0,0.004028
1,0-11,0.001445,200.047775,1.000239,0.000262,200.041168,1.000206,0.000272,0.000360,0.001204,...,0.000802,22.0,1124.0,2.045455,0.000510,11.0,829.0,2.090909,0,0.004025
2,0-16,0.002169,187.913849,0.999542,0.000864,187.939819,0.999680,0.000862,-0.002074,0.002369,...,0.001575,18.0,1691.0,2.833333,0.001048,10.0,1087.0,3.400000,0,0.004021
3,0-31,0.002195,119.859779,0.998832,0.000757,119.835945,0.998633,0.000656,-0.002828,0.002574,...,0.001090,10.0,1561.0,4.700000,0.000802,3.0,514.0,3.666667,0,0.004025
4,0-62,0.001747,175.932861,0.999619,0.000258,175.934250,0.999626,0.000317,-0.000002,0.001894,...,0.000498,14.0,1458.0,4.428571,0.000395,6.0,162.0,3.666667,0,0.004025


In [27]:
df_test.head()

,stock_id,time_id,row_id,wap_sum,wap_mean,wap_std,wap2_sum,wap2_mean,wap2_std,log_return_sum,...,trade_order_count_mean,trade_log_return_realized_volatility_200,trade_seconds_in_bucket_count_unique_200,trade_size_sum_200,trade_order_count_mean_200,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_count_unique_400,trade_size_sum_400,trade_order_count_mean_400,stock_id_target_enc
0,0,4,0-4,3.001215,1.000405,0.00017,3.00165,1.00055,0.000153,0.000294,...,3.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
1,0,32,0-32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
2,0,34,0-34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028


# LightGBM

In [28]:
import lightgbm as lgbm
from bayes_opt import BayesianOptimization

In [29]:
# Transform stock_id to integer
df_train['stock_id'] = df_train['stock_id'].astype(int)
df_test['stock_id'] = df_test['stock_id'].astype(int)

### Cross Validation

In [30]:
# Split features and target
X = df_train.drop(['row_id','target'], axis=1)
y = df_train['target']

In [31]:
# Hyper-parameter tuning (Bayesian Optimization)
def bayes_parameter_opt_lgb(X, y, init_round = 15, opt_round = 25, n_folds = 3, random_seed = 6, n_estimators = 10000, output_process = False):
    # prepare data
    train_data = lgbm.Dataset(data = X, label = y, free_raw_data = False)
    # parameters
    def lgb_eval(learning_rate,num_leaves, feature_fraction, bagging_fraction, max_depth, max_bin, min_data_in_leaf,min_sum_hessian_in_leaf,subsample):
        params = {'application':'binary', 'metric':'auc'}
        params['learning_rate'] = max(min(learning_rate, 1), 0)
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['max_bin'] = int(round(max_depth))
        params['min_data_in_leaf'] = int(round(min_data_in_leaf))
        params['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
        params['subsample'] = max(min(subsample, 1), 0)
        
        cv_result = lgbm.cv(params, train_data, nfold = n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])
     
    lgbBO = BayesianOptimization(lgb_eval, {'learning_rate': (0.01, 1.0), # recommended to use smaller learning_rate with larger num_iterations
                                            'num_leaves': (24, 30000), # num_leaves = 2^(max_depth)
                                            'feature_fraction': (0.1, 0.9), # randomly select a subset of features on each iteration (tree) (%) -> speed + overfit
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 30),
                                            'max_bin':(20,90),
                                            'min_data_in_leaf': (20, 1000),
                                            'min_sum_hessian_in_leaf':(0,100),
                                           'subsample': (0.01, 1.0)}, random_state=200)

    
    #n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
    #init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
    
    lgbBO.maximize(init_points = init_round, n_iter = opt_round)
    
    model_auc=[]
    for model in range(len( lgbBO.res)):
        model_auc.append(lgbBO.res[model]['target'])
    
    # return best parameters
    return lgbBO.res[pd.Series(model_auc).idxmax()]['target'],lgbBO.res[pd.Series(model_auc).idxmax()]['params']

opt_params = bayes_parameter_opt_lgb(X, y, init_round = 5, opt_round = 10, n_folds = 3, random_seed = 6,n_estimators = 10000)

|   iter    |  target   | baggin... | featur... | learni... |  max_bin  | max_depth | min_da... | min_su... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] bagging_fraction is set=0.9895264513703341, subsample=0.4615201756036703 will be ignored. Current value: bagging_fraction=0.9895264513703341


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [78]:
# Optimized parameters
opt_params[1]["num_leaves"] = int(round(opt_params[1]["num_leaves"]))
opt_params[1]['max_depth'] = int(round(opt_params[1]['max_depth']))
opt_params[1]['min_data_in_leaf'] = int(round(opt_params[1]['min_data_in_leaf']))
opt_params[1]['max_bin'] = int(round(opt_params[1]['max_bin']))
opt_params[1]['objective'] = 'binary'
opt_params[1]['metric'] = 'auc'
opt_params[1]['is_unbalance'] = True
opt_params[1]['boost_from_average'] = False
opt_params = opt_params[1]
opt_params

NameError: name 'opt_params' is not defined

In [32]:
# To calculate RMSPE
def rmspe(y_true, y_pred):
    return (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

# Early stop with RMSPE
def feval_RMSPE(preds, lgbm_train): #Customized evaluation function
    labels = lgbm_train.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False

seed = 42
params = {
      'objective': 'rmse', 
      'metric': 'rmse', 
      'boosting_type': 'gbdt',
      'learning_rate': 0.01,        
        'num_leaves': 27293,
        'feature_fraction': 0.2812379369053784,
        'min_sum_hessian_in_leaf': 35.74236812873617,
        'bagging_fraction': 0.9895264513703341,
        'max_bin': 50,
        'min_data_in_leaf': 23,
        'max_depth': 24,
        'seed': seed,
        'feature_fraction_seed': seed,
        'bagging_seed': seed,
        'drop_seed': seed,
        'data_random_seed': seed,
        'verbosity': -1,
        'n_jobs': -1
  }

In [33]:
from sklearn.model_selection import KFold
# Create a KFold object
kf = KFold(n_splits = 20, random_state = 19901028, shuffle = True)
oof = pd.DataFrame()                 # out-of-fold result
models = []                          # models
scores = 0.0                         # validation score

In [ ]:
# Iterate through each fold
#%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):

    print("Fold", fold + 1)
    
    # Create dataset
    X_train, y_train = X.loc[trn_idx], y[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y[val_idx]
    
    # RMSPE weights
    train_weights = 1/np.square(y_train)
    lgbm_train = lgbm.Dataset(X_train, y_train, weight = train_weights)

    valid_weights = 1/np.square(y_valid)
    lgbm_valid = lgbm.Dataset(X_valid, y_valid, reference = lgbm_train, weight = valid_weights)
    
    # Model 
    model = lgbm.train(params = params,
                      train_set = lgbm_train,
                      valid_sets = [lgbm_train, lgbm_valid],
                      num_boost_round = 10000,
                      early_stopping_rounds = 100, # rule of thumb is to have it at 10% of your num_iterations.
                      feval = feval_RMSPE,
                      verbose_eval = 200, # the eval metric on the valid set is printed at every 100 boosting stage. 
                      categorical_feature = ['stock_id']                
                     )
    
    # validation 
    y_pred = model.predict(X_valid, num_iteration = model.best_iteration)

    RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred),3)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')

    #keep scores and models
    scores += RMSPE / 20
    models.append(model)
    print("*" * 100)

Fold 1


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
[200]	training's rmse: 0.000301211	training's RMSPE: 0.32313	valid_1's rmse: 0.000446118	valid_1's RMSPE: 0.36688
[400]	training's rmse: 0.000200807	training's RMSPE: 0.29017	valid_1's rmse: 0.000431731	valid_1's RMSPE: 0.34639
[600]	training's rmse: 0.000154818	training's RMSPE: 0.28167	valid_1's rmse: 0.000430215	valid_1's RMSPE: 0.34387
[800]	training's rmse: 0.000121613	training's RMSPE: 0.27757	valid_1's rmse: 0.000429778	valid_1's RMSPE: 0.34351
[1000]	training's rmse: 9.88786e-05	training's RMSPE: 0.27549	valid_1's rmse: 0.00042958	valid_1's RMSPE: 0.34324
[1200]	training's rmse: 8.21046e-05	training's RMSPE: 0.27425	valid_1's rmse: 0.000429494	valid_1's RMSPE: 0.34295
Early stopping, best iteration is:
[1241]	training's rmse: 7.904e-05	training's RMSPE: 0.27405	valid_1's rmse: 0.000429478	valid_1's RMSPE: 0.34287
Performance of the　prediction: , RMSPE: 0.343
***********************************************************

/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
[200]	training's rmse: 0.000300968	training's RMSPE: 0.32389	valid_1's rmse: 0.000452777	valid_1's RMSPE: 0.34857
[400]	training's rmse: 0.000200709	training's RMSPE: 0.29067	valid_1's rmse: 0.000439641	valid_1's RMSPE: 0.32909
[600]	training's rmse: 0.000155189	training's RMSPE: 0.28211	valid_1's rmse: 0.000438241	valid_1's RMSPE: 0.32679
[800]	training's rmse: 0.000122427	training's RMSPE: 0.27796	valid_1's rmse: 0.000437679	valid_1's RMSPE: 0.32641
Early stopping, best iteration is:
[826]	training's rmse: 0.000119205	training's RMSPE: 0.27761	valid_1's rmse: 0.000437641	valid_1's RMSPE: 0.32639
Performance of the　prediction: , RMSPE: 0.326
****************************************************************************************************
Fold 3


...

Training until validation scores don't improve for 100 rounds
[200]	training's rmse: 0.000301245	training's RMSPE: 0.32359	valid_1's rmse: 0.00044531	valid_1's RMSPE: 0.35459
[400]	training's rmse: 0.000200686	training's RMSPE: 0.29031	valid_1's rmse: 0.000430892	valid_1's RMSPE: 0.33518
[600]	training's rmse: 0.000155032	training's RMSPE: 0.28178	valid_1's rmse: 0.000429224	valid_1's RMSPE: 0.33262
[800]	training's rmse: 0.00012197	training's RMSPE: 0.27758	valid_1's rmse: 0.000428711	valid_1's RMSPE: 0.33214
Early stopping, best iteration is:
[842]	training's rmse: 0.000116515	training's RMSPE: 0.27702	valid_1's rmse: 0.000428608	valid_1's RMSPE: 0.33206
Performance of the　prediction: , RMSPE: 0.332
****************************************************************************************************
Fold 7


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
[200]	training's rmse: 0.000301101	training's RMSPE: 0.32382	valid_1's rmse: 0.000451599	valid_1's RMSPE: 0.34707
[400]	training's rmse: 0.000200478	training's RMSPE: 0.29037	valid_1's rmse: 0.000438557	valid_1's RMSPE: 0.32915
[600]	training's rmse: 0.00015546	training's RMSPE: 0.28181	valid_1's rmse: 0.00043711	valid_1's RMSPE: 0.32714
[800]	training's rmse: 0.000122729	training's RMSPE: 0.27756	valid_1's rmse: 0.000436445	valid_1's RMSPE: 0.32674
Early stopping, best iteration is:
[816]	training's rmse: 0.000120744	training's RMSPE: 0.27733	valid_1's rmse: 0.000436433	valid_1's RMSPE: 0.32672
Performance of the　prediction: , RMSPE: 0.327
****************************************************************************************************
Fold 8


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
[200]	training's rmse: 0.000301155	training's RMSPE: 0.32365	valid_1's rmse: 0.000448644	valid_1's RMSPE: 0.35187
[400]	training's rmse: 0.000200412	training's RMSPE: 0.29036	valid_1's rmse: 0.000436179	valid_1's RMSPE: 0.33399
[600]	training's rmse: 0.000154938	training's RMSPE: 0.28179	valid_1's rmse: 0.000434985	valid_1's RMSPE: 0.33168
[800]	training's rmse: 0.00012201	training's RMSPE: 0.27764	valid_1's rmse: 0.000434467	valid_1's RMSPE: 0.33134
[1000]	training's rmse: 9.98509e-05	training's RMSPE: 0.27557	valid_1's rmse: 0.000434294	valid_1's RMSPE: 0.33126
Early stopping, best iteration is:
[1063]	training's rmse: 9.41796e-05	training's RMSPE: 0.27511	valid_1's rmse: 0.00043428	valid_1's RMSPE: 0.33122


In [ ]:
scores

# Test set

In [ ]:
df_test.columns

In [ ]:
df_train.columns

In [ ]:
y_pred = df_test[['row_id']]
X_test = df_test.drop(['time_id', 'row_id'], axis = 1)

In [ ]:
X_test

In [ ]:
target = np.zeros(len(X_test))

# light gbm models
for model in models:
    pred = model.predict(X_test[X_valid.columns], num_iteration=model.best_iteration)
    target += pred / len(models)

In [ ]:
y_pred = y_pred.assign(target = target)

In [ ]:
y_pred

In [ ]:
y_pred.to_csv('submission.csv',index = False)